<a href="https://colab.research.google.com/github/kyo106/CD/blob/master/%E3%80%8CJPY_COIN_240607%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
send_line_notify("測試訊息", token)


Line Notify 回應: {"status":200,"message":"ok"}


In [ ]:
import time
from urllib import request, parse
from bs4 import BeautifulSoup as sp
from datetime import datetime
import pytz

# 設定價格
set_price_jpy = "0.2111"  # 日幣到價通知
set_price_usd = "32.2"  # 美金到價通知
# Line Notify Token
token = 'JqPcyMepDMsPay0VZ5PdESn9RShAzHIJQgEwCLMl0WN'

def send_line_notify(message, token):
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"message": message}
    data = parse.urlencode(data).encode()
    req = request.Request(url, data=data, headers=headers)
    with request.urlopen(req) as response:
        res = response.read().decode("utf-8")
    print(f"Line Notify 回應: {res}")

# standard function
def standard(jpy_out, usd_out, note):
    if jpy_out == set_price_jpy or usd_out == set_price_usd:
        send_line_notify(note, token)  # 將 note 發送
    else:
        print(f"未達設定價格: 日幣當前價格 {jpy_out}，設定價格 {set_price_jpy}")
        print(f"未達設定價格: 美金當前價格 {usd_out}，設定價格 {set_price_usd}")

# main function
def main():
    tz = pytz.timezone('Asia/Taipei')
    while True:
        current_time = datetime.now(tz)
        current_hour = current_time.hour
        if 9 <= current_hour < 17:  # 只在早上9點到下午5點之間檢查
            local_time = current_time.ctime()
            url = "https://rate.bot.com.tw/xrt?Lang=zh-TW"
            with request.urlopen(url) as response:
                data = response.read().decode("utf-8")
            root = sp(data, "html.parser")
            jpy_in = root.find_all("td")[78].text.strip()
            jpy_out = root.find_all("td")[79].text.strip()
            usd_in = root.find_all("td")[2].text.strip()
            usd_out = root.find_all("td")[3].text.strip()
            s1 = (f"\njpy\n銀行買進: {jpy_in}\n銀行賣出: {jpy_out}"
                  f"\nusd\n銀行買進: {usd_in}\n銀行賣出: {usd_out}\nBy Bruno")  # 在銀行賣出價格後加上 "By Bruno"
            note = local_time + s1
            print(f"當前時間: {local_time}")
            print(f"銀行買進價格: 日幣 {jpy_in}, 日幣賣出價格: {jpy_out}")
            print(f"銀行買進價格: 美金 {usd_in}, 美金賣出價格: {usd_out}")
            standard(jpy_out, usd_out, note)
        else:
            print("非檢查時間段內，暫停檢查")
        time.sleep(1800)  # 每半小時檢查一次

if __name__ == "__main__":
    main()


當前時間: Fri Jun  7 15:52:00 2024
銀行買進價格: 日幣 0.1983, 日幣賣出價格: 0.2111
銀行買進價格: 美金 32.52, 美金賣出價格: 32.2
Line Notify 回應: {"status":200,"message":"ok"}
